In [18]:
import re
from num2words import num2words
def goal_translate(goals):
    goal_trans = ''
    sub_goal = ''
    for goal in goals:
        act = re.findall(r'\w+\b', goal)
        #print(act)
        if len(act)==2:
            sub_goal = '{} the {}'.format(act[0],act[1])
        if len(act)==4:
            sub_goal = 'put {} {} {} {}'.format(num2words(act[3]),act[1], act[0], act[2])
        goal_trans = goal_trans + sub_goal + ', '
    goal_trans = goal_trans.lower()
    goal_trans = goal_trans[:-2] + '.'
    return goal_trans

In [19]:
goals = ['INSIDE (cutleryfork, dishwasher): 3','ON (wineglass, sink): 2','INSIDE (waterglass, dishwasher): 1',
         'CLOSE (dishwasher)','TURNON (dishwasher)']
goal_translate(goals)

['INSIDE', 'cutleryfork', 'dishwasher', '3']
['ON', 'wineglass', 'sink', '2']
['INSIDE', 'waterglass', 'dishwasher', '1']
['CLOSE', 'dishwasher']
['TURNON', 'dishwasher']


'put three cutleryfork inside dishwasher, put two wineglass on sink, put one waterglass inside dishwasher, close the dishwasher, turnon the dishwasher.'

In [45]:
from transformers import GPT2Tokenizer, GPT2Model
import tensorflow as tf
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "I have a dream"
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
output = model(**encoded_input)
print(output.last_hidden_state.detach().numpy())

{'input_ids': tensor([[  40,  423,  257, 4320]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
[[[-0.07962783 -0.06535359 -0.08424436 ... -0.14416133 -0.04557567
    0.01425   ]
  [-0.19016495 -0.17212972 -0.79750437 ... -0.17365916  0.6244487
    0.30738056]
  [ 0.03623195  0.12202189 -0.21445212 ... -0.38196504  0.34530863
    0.07974897]
  [-0.1296617  -0.6143206  -1.5262644  ... -0.22204967  0.13208918
    0.08905733]]]


In [46]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')  # or any other checkpoint
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
text_index = tokenizer.encode('I have a dream')
print(text_index)
vector = word_embeddings[text_index,:]
print(vector)

AttributeError: 'GPT2Model' object has no attribute 'transformer'

In [3]:
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')
generator("kitchencabinet", max_length=30, num_return_sequences=5)

RuntimeError: Failed to import transformers.models.gpt2.modeling_tf_gpt2 because of the following error (look up to see its traceback):
DLL load failed while importing defs: The specified procedure could not be found.

In [44]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch as tc
modelGPT = GPT2Model.from_pretrained('gpt2')
modelGPT(inputs_embeds=tc.FloatTensor(vector.detach().numpy())).last_hidden_state

tensor([[-0.0796, -0.0654, -0.0842,  ..., -0.1442, -0.0456,  0.0142],
        [-0.1902, -0.1721, -0.7975,  ..., -0.1737,  0.6244,  0.3074],
        [ 0.0362,  0.1220, -0.2145,  ..., -0.3820,  0.3453,  0.0797],
        [-0.1297, -0.6143, -1.5263,  ..., -0.2220,  0.1321,  0.0891]],
       grad_fn=<ViewBackward0>)

create word embeddings and tokenizer

In [92]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from keras.models import Sequential, Model, Input
from keras.layers import Dense, ReLU, Concatenate, Lambda
import numpy as np

def get_word_embeddings(lm):
    model = GPT2LMHeadModel.from_pretrained(lm)
    word_embeddings = model.transformer.wte.weight
    return word_embeddings

def get_tokenizer(lm):
    tokenizer = GPT2Tokenizer.from_pretrained(lm)
    return tokenizer

gpt2_word_embeddings = get_word_embeddings('gpt2')
gpt2_tokenizer = get_tokenizer('gpt2')

embed goal

In [69]:
def get_goal_embeddings(word_embeddings, tokenizer, goal):
    goal_token = tokenizer.encode(goal)
    goal_embeddings = word_embeddings[goal_token,:].detach().numpy()
    return goal_embeddings

gpt2_goal_embeddings = get_goal_embeddings(gpt2_word_embeddings, gpt2_tokenizer, 'put one apple on the table')

embed history

In [70]:
def get_history_embeddings(word_embeddings, tokenizer, history):
    history_token = tokenizer.encode(history)
    history_embeddings = word_embeddings[history_token,:].detach().numpy()
    return history_embeddings

gpt2_history_embeddings = get_history_embeddings(gpt2_word_embeddings, gpt2_tokenizer, 'I have walked to the kitchen, grabbed the apple')

embed object

In [94]:
# embed name
def get_name_embeddings(name):
    name_token = gpt2_tokenizer.encode(name)
    name_embeddings = gpt2_word_embeddings[name_token,:].detach().numpy()
    name_embeddings = np.mean(name_embeddings, axis=0)
    return name_embeddings

gpt2_name_embeddings = get_name_embeddings('kitchen oven')

def get_state_model():
    model = Sequential()
    model.add(Dense(32, input_dim=6))
    return model

gpt2_state_model = get_state_model()
gpt2_state_embeddings = gpt2_state_model.predict(np.array([[1, 0, 0, 1, 0, 0]]))

def get_position_model():
    model = Sequential()
    model.add(Dense(64, input_dim=6))
    model.add(ReLU)
    model.add(Dense(64))
    return model

gpt2_position_model = get_state_model()
gpt2_position_embeddings = gpt2_position_model.predict(np.array([[0.4, 0.2, 0.4, 0.1, 0.02, 0.03]]))

def get_object_model(word_embeddings, tokenizer, state_model, position_model, name, state, position):
    name_token = tokenizer.encode(name)
    name_embeddings = word_embeddings[name_token,:].detach().numpy()
    name_embeddings = np.mean(name_embeddings, axis=0)

    state_embeddings = state_model.predict(state)

    position_embeddings = position_model.predict(position)

    merged = np.concatenate([name_embeddings, state_embeddings, position_embeddings])

    merged = Dense(768)(merged)

    input = Input(shape=(864, 1))
    model = Model(input, merged)
    return model

def get_object_model_new():
    name_input = Input(shape=(1, 768))
    state_input = Input(shape=(1, 6))
    position_input = Input(shape=(1, 6))

    name_embeddings = Lambda(lambda x: x)(name_input)

    state_embeddings = Dense(64)(state_input)

    position_embeddings = Dense(64)(position_input)
    position_embeddings = ReLU()(position_embeddings)
    position_embeddings = Dense(64)(position_embeddings)

    merged = Concatenate()([name_embeddings, state_embeddings, position_embeddings])
    merged = Dense(768)(merged)

    model = Model([name_input, state_input, position_input], merged)
    return model

object_model = get_object_model_new()
print(object_model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_33 (InputLayer)          [(None, 1, 6)]       0           []                               
                                                                                                  
 dense_55 (Dense)               (None, 1, 64)        448         ['input_33[0][0]']               
                                                                                                  
 input_31 (InputLayer)          [(None, 1, 768)]     0           []                               
                                                                                                  
 input_32 (InputLayer)          [(None, 1, 6)]       0           []                               
                                                                                            

In [ ]:
print(1)